# LangChain: Models, Prompts and Output Parsers

1. 使用ollama下载deepseek-r1:32b  
2. 后台启动ollama服务： nohup ollama serve  >/dev/null 2>&1 &  
3. 我用的是3090, 共24G显存，占用了21G。

### Model

In [41]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI(
    model_name='deepseek-r1:32b',
    openai_api_base="http://127.0.0.1:11434/v1",
    openai_api_key="EMPTY",
    streaming=True
)
chat

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x703363aaf610>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x703363ad9630>, root_client=<openai.OpenAI object at 0x703363a3d870>, root_async_client=<openai.AsyncOpenAI object at 0x703363aaf670>, model_name='deepseek-r1:32b', model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='http://127.0.0.1:11434/v1', streaming=True)

### Prompt template

In [49]:
template_string = """将由三个反引号分隔的文本\
翻译成{style}风格。\
文本：```{text}```
"""

In [50]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [51]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='将由三个反引号分隔的文本翻译成{style}风格。文本：```{text}```\n')

In [52]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [54]:
customer_style = """山东方言，\
以平静和尊重的语气
"""

In [55]:
customer_email = """
唉，我非常生气，因为我的搅拌机盖子飞了出去，把厨房的墙壁溅满了果昔！更糟糕的是，保修单不包括清洁厨房的费用。我现在就需要你的帮助，伙计！
"""

In [56]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [57]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [58]:
print(customer_messages[0])

content='将由三个反引号分隔的文本翻译成山东方言，以平静和尊重的语气\n风格。文本：```\n唉，我非常生气，因为我的搅拌机盖子飞了出去，把厨房的墙壁溅满了果昔！更糟糕的是，保修单不包括清洁厨房的费用。我现在就需要你的帮助，伙计！\n```\n' additional_kwargs={} response_metadata={}


In [59]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

In [60]:
print(customer_response.content)

<think>
嗯，首先我要处理用户的请求，他们想要将一段中文文本翻译成山东方言，并且语气要平静和尊重。这段话原本表达的是说话者对搅拌机盖子飞出去导致墙壁弄脏果昔，同时保修单不包括清洁费用的愤怒情绪。我的任务是转换语气。

我需要确保在保持原意的同时，语气变得平和一些。这可能意味着用词上避免显得过于激烈或者抱怨，而是换成更中立或稍微轻松点的表达方式。

接着，我要考虑山东方言的特点，这样翻译才能更加地道。比如“伙计”可以替换为更具地方特色的词汇，让整个句子听起来更有地域特色。

另外，我还要注意保持条理清晰，让用户容易理解转换后的语言，并且传达出平静而礼貌的感觉。这可能涉及到调整一些句式结构，使其在方言中更自然流畅。

最后，我会检查整体语气是否符合用户的要求，确保没有遗漏任何关键信息，并且整个翻译看起来自然、真实，能够让说话者以一种更具亲和力的方式表达他们的困扰。
</think>

唉，真是让人头疼，我的搅拌机盖子飞出去了，把厨房的墙上溅了一大片果昔。更麻烦的是保修单里不包括清理 Kitchen 的费用。现在我非常需要你的帮助，老哥！


In [61]:
service_reply = """嘿，伙计，保修条款可不包括你厨房的清洁费用，\
这事儿怪不得别人，只因你使用搅拌机的时候忘了盖上盖子，\
这才导致了这场混乱。\
只能自认倒霉啦！再见喽！
"""

In [77]:
service_style_pirate = """\
一种礼貌的语气，\
用上海方言表达\
"""

In [78]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

将由三个反引号分隔的文本翻译成一种礼貌的语气，用上海方言表达风格。文本：```嘿，伙计，保修条款可不包括你厨房的清洁费用，这事儿怪不得别人，只因你使用搅拌机的时候忘了盖上盖子，这才导致了这场混乱。只能自认倒霉啦！再见喽！
```



In [79]:
service_response = chat(service_messages)
print(service_response.content)

<think>
好，我现在得帮用户把一段文本翻译成上海方言，语气要礼貌。首先看看原文：“嘿，伙计，保修条款可不包括你厨房的清洁费用，这事儿怪不得别人，只因你使用搅拌机的时候忘了盖上盖子，这才导致了这场混乱。只能自认倒霉啦！再见喽！”这段话比较直接，语气有点责备的感觉。

用户希望翻译成上海方言，但要有礼貌。那我得分析一下原文的结构和信息重点在哪里。首先，“嘿，伙计”可以改成更亲切的称呼，比如“您”。然后，“保修条款可不包括你厨房的清洁费用”这部分需要保留原意，同时用沪语表达。

接下来要处理的是责任归属。在原文中说“这事儿怪不得别人”，可以转成“这个情况，我们建议以后使用的时候要小心一些哦”。这样既保持了说明问题的原因，又显得更有建设性。

关于搅拌机没盖盖子导致的混乱，我可以解释为操作上的失误，并强调这是为了机器的正常运行和使用安全。“请记得每次使用后都要把盖子盖好，这样不仅能延长使用寿命，也能保证使用的安全性。”

最后，结尾部分要礼貌道别。可以加上“如果有任何其他问题或需要帮助的地方，随时联系我们，我们会尽力协助您的。”这样不仅礼貌还提供了进一步的帮助途径。

整体上来回调整表达，确保用词亲切，语气柔和，同时准确传达原信息和建议用户以后注意使用细节。沪语的转换也要自然，让人听起来既熟悉又专业。
</think>

侬好呀！这里跟您说一声啊，咱这保修条款里面不包含厨房清洁费用哦！这情况嘛，主要是您在用搅拌机的时候忘记盖上盖子了才会有这样的状况出现啊！以后使用的时候要小心一些哦！记得每次使用后都要把盖子盖好了，这样不仅可以延长使用寿命，还能保证使用的安全性呢！

如果以后还有什么问题或者需要帮助的地方，尽管联系我们哈！我们会全力帮忙的。再见啦！

（注：这段话在保留原意的基础上，采用了更加温和、贴心的沪语表达方式，并且避免了可能带来的冲突感。）


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [80]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [81]:
customer_review = """\
这款吹叶机真的很棒。它有四种设置：\
烛火吹拂、微风轻拂、风城强风和龙卷风。\
它在两天内就到了，正好赶上我妻子的\
周年纪念礼物。\
我觉得我的妻子非常喜欢它，甚至惊讶得说不出话来。\
到目前为止，只有我在使用它，每隔一个早上我都会用它\
清理我们草坪上的落叶。\
它的价格比其他吹叶机稍贵一些，但我认为为了这些额外的功能，这是值得的。
"""

review_template = """\
对于以下文本，请提取以下信息：

gift: 该物品是否作为礼物购买给其他人？如果是，回答 True；如果不是或未知，回答 False。

delivery_days: 产品送达需要多少天？如果找不到此信息，输出 -1。

price_value: 提取任何关于价值或价格的句子，并将它们作为逗号分隔的 Python 列表输出。

以 JSON 格式输出，包含以下键：
gift
delivery_days
price_value

文本：{text}
"""

In [82]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='对于以下文本，请提取以下信息：\n\ngift: 该物品是否作为礼物购买给其他人？如果是，回答 True；如果不是或未知，回答 False。\n\ndelivery_days: 产品送达需要多少天？如果找不到此信息，输出 -1。\n\nprice_value: 提取任何关于价值或价格的句子，并将它们作为逗号分隔的 Python 列表输出。\n\n以 JSON 格式输出，包含以下键：\ngift\ndelivery_days\nprice_value\n\n文本：{text}\n'), additional_kwargs={})]


In [83]:
messages = prompt_template.format_messages(text=customer_review)
#chat = ChatOpenAI(temperature=0.0, model=llm_model)
response = chat(messages)
print(response.content)

<think>
好的，我现在需要处理这个用户的查询。用户给了一个任务，要求从一段文本中提取特定信息，并按照指定的JSON格式输出。首先，我要仔细阅读用户的要求，确保我理解每个部分的需求。

文本内容是这样的：吹叶机很好，有四种设置，两天送达，作为妻子周年纪念礼物购买。价格比其他吹叶机贵一些，但认为物有所值。那么，我需要提取三个字段：gift, delivery_days 和 price_value。

首先处理gift字段。用户问是否为礼物购买给他人，如果是，回答True，否则False或未知时返回False。文本中明确提到“正好赶上我妻子的周年纪念礼物”，所以这里应该是True。

接下来是delivery_days，要求提取送达天数。文本里说“两天内就到了”，所以是2天。如果找不到信息则输出-1，但这里是有的，因此填写2。

最后是price_value，需要提取所有关于价格或价值的句子，并以逗号分隔为列表。原文中有两个相关句子：“它的价格比其他吹叶机稍贵一些”和“但我认为为了这些额外的功能，这是值得的”。所以这两个都需要包含进去。

现在将这三个部分整合成JSON格式。确保键名正确，值对应准确。特别是price_value要用逗号分隔，这里用Python列表的形式呈现比较合适。

检查文本是否有其他信息可能影响结果：比如礼物是否明确指向他人？是的，给妻子作为周年纪念，所以True没错。送达时间两天没错。价格部分也提取了所有相关信息。

最后，将这些整理成一个JSON结构，确保语法正确，没有遗漏或错误。
</think>

```json
{
  "gift": True,
  "delivery_days": 2,
  "price_value": [
    "它的价格比其他吹叶机稍贵一些",
    "但我认为为了这些额外的功能，这是值得的"
  ]
}
```


In [84]:
type(response.content)

str

In [85]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

### Parse the LLM output string into a Python dictionary

In [86]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [87]:
gift_schema = ResponseSchema(name="gift",
                             description="该物品是否作为礼物购买给其他人？如果是，回答 True；如果不是或未知，回答 False。")

delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="产品送达需要多少天？如果找不到此信息，输出 -1。")

price_value_schema = ResponseSchema(name="price_value",
                                    description="提取任何关于价值或价格的句子，并将它们作为逗号分隔的 Python 列表输出。")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [88]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [89]:
format_instructions = output_parser.get_format_instructions()

In [90]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // 该物品是否作为礼物购买给其他人？如果是，回答 True；如果不是或未知，回答 False。
	"delivery_days": string  // 产品送达需要多少天？如果找不到此信息，输出 -1。
	"price_value": string  // 提取任何关于价值或价格的句子，并将它们作为逗号分隔的 Python 列表输出。
}
```


In [91]:
review_template_2 = """\
对于以下文本，请提取以下信息：

gift: 该物品是否作为礼物购买给其他人？如果是，回答 True；如果不是或未知，回答 False。

delivery_days: 产品送达需要多少天？如果找不到此信息，输出 -1。

price_value: 提取任何关于价值或价格的句子，并将它们作为逗号分隔的 Python 列表输出。

文本：{text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [92]:
print(messages[0].content)

对于以下文本，请提取以下信息：

gift: 该物品是否作为礼物购买给其他人？如果是，回答 True；如果不是或未知，回答 False。

delivery_days: 产品送达需要多少天？如果找不到此信息，输出 -1。

price_value: 提取任何关于价值或价格的句子，并将它们作为逗号分隔的 Python 列表输出。

文本：这款吹叶机真的很棒。它有四种设置：烛火吹拂、微风轻拂、风城强风和龙卷风。它在两天内就到了，正好赶上我妻子的周年纪念礼物。我觉得我的妻子非常喜欢它，甚至惊讶得说不出话来。到目前为止，只有我在使用它，每隔一个早上我都会用它清理我们草坪上的落叶。它的价格比其他吹叶机稍贵一些，但我认为为了这些额外的功能，这是值得的。


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // 该物品是否作为礼物购买给其他人？如果是，回答 True；如果不是或未知，回答 False。
	"delivery_days": string  // 产品送达需要多少天？如果找不到此信息，输出 -1。
	"price_value": string  // 提取任何关于价值或价格的句子，并将它们作为逗号分隔的 Python 列表输出。
}
```



In [93]:
response = chat(messages)

In [94]:
print(response.content)

<think>
今天，我收到了用户的查询，要求从一段文本中提取特定的信息。首先，我需要仔细阅读用户的问题，明确每个提取点的要求。

用户提供的文本是关于一款吹叶机的评论。我需要解析其中的内容，提取三个关键信息：gift、delivery_days和price_value。具体来说：

1. **gift**：判断该物品是否作为礼物购买给其他人。如果是，返回True；否则或未知时返回False。
   
   文本中提到“正好赶上我妻子的周年纪念礼物”，这明确表明吹叶机是作为礼物送给妻子的，所以这里应该标记为True。

2. **delivery_days**：提取产品送达所需的天数，如果找不到相关信息，则输出-1。
   
   在文本中有句话“它在两天内就到了”，直接说明了送货时间是两天。因此，这个值应该是2。

3. **price_value**：提取关于价格或价值的句子，并生成一个逗号分隔的Python列表。
   
   文本中提到两处与价格相关的内容：“它的价格比其他吹叶机稍贵一些”和“我认为为了这些额外的功能，这是值得的”。这两个句子都需要被提取出来作为price_value的内容。

检查文本时，确保没有遗漏任何相关信息。例如，在关于礼物的部分，文本明确指出了购买动机是送给妻子作为纪念礼物，这一点很清晰。送货时间也是直接给出的两天，不存在模棱两可的情况。至于价格部分，虽然有两个相关的句子，但其中第二个句子更多是在表达个人认为物有所值的观点，而不仅仅是描述价格，所以是否需要纳入可能值得考虑。

不过，根据用户的要求是提取任何涉及价格或价值的句子，因此即使这些句子不是直接给出价格数字，只要是关于价值评估的内容也应该被包含进去。这样就确保了提取信息的全面性。

最后，将这些提取的信息按照指定的JSON格式组织起来，并确认语法正确。在生成过程中，特别注意逗号和引号的使用是否正确，避免出现解析错误。

总的来说，整个过程需要细致地梳理文本内容，准确识别出每个关键点，并严格按照用户的要求进行信息抽取和格式化输出。
</think>

```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": "它的价格比其他吹叶机稍贵一些, 我认为为了这些额外的功能，这是值得的"
}
```


In [95]:
output_dict = output_parser.parse(response.content)

In [96]:
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': '它的价格比其他吹叶机稍贵一些, 我认为为了这些额外的功能，这是值得的'}

In [97]:
type(output_dict)

dict

In [98]:
output_dict.get('delivery_days')

'2'

Reminder: Download your notebook to you local computer to save your work.